Before we do any work, we need to import several functions from cdapython:
- `Q` and `query` which power the search
- `columns` which lets us view entity field names
- `unique_terms` which lets view entity field contents

We're also asking cdapython to report it's version so we can be sure we're using the one we mean to.

In [1]:
from cdapython import Q, columns, unique_terms,query, constantVariables
import cdapython
print(cdapython.__file__)
print(cdapython.__version__)
integration_host = "http://35.192.60.10:8080/"
Q.set_host_url(integration_host)

/Users/amanda.charbonneau/github/cda-python/cdapython/__init__.py
2022.5.9


CDA data comes from three sources:
- The [Proteomic Data Commons](https://proteomic.datacommons.cancer.gov/pdc/) (PDC)
- The [Genomic Data Commons](https://gdc.cancer.gov/) (GDC)
- The [Imaging Data Commons](https://datacommons.cancer.gov/repository/imaging-data-commons) (IDC)

The CDA makes this data searchable in four endpoints:

- `subject`: Specific, unique, individuals
- `research_subject`: Study-individual aggregate entities. A Subject who was part of three studies will appear as three ResearchSubjects
- `specimen`: Samples taken from individual
- `file`: Data about Subjects, ResearchSubjects, Specimens, and their associated information


If you are looking to build a cohort of distinct individuals who meet some criteria, search by `Subject`. If you want to build a cohort, but are particularly interested in studies rather than the participates per se, search by `ResearchSubject`. If you are looking for biosamples that can be ordered or a specfic format of information (for e.g. histological slides) start with `Specimen`. If you are primarily looking for files you can reuse for your own analysis, start with `File`.

In CDA search, these concepts can also be strung together, so you can look specifically for Specimen Files, or ResearchSubject Specimens. In all cases, any search can use any metadata field, the only difference between search types is what type of data you return by default.


## Basic search with endpoints

Let's try a broad search of the CDA to see what information exists about cancers that were first diagnosed in the brain. To run this simple search, we would first construct a query in `Q` and save it to a variable `myquery`:

In [2]:
myquery = Q('ResearchSubject.primary_diagnosis_site = "brain"')



<div style="background-color:#6ce6b9;color:black;padding:20px;">
<h3>Where did those terms come from?</h3>
    
If you aren't sure how we knew what terms to put in our search, please refer back to the <a href="../SearchTerms">What search terms are available?</a> notebook. 
</div>

### subject
Now we can use that query to search any of information types. Let's start by looking at what Subjects meet our criteria. To do that, we will send our query to the subject endpoint, then ask for it to run:

In [3]:
subjectresults = myquery.subject.run()

Getting results from database

Total execution time: 4638 ms


We saved the output in a variable `subjectresults`, so we don't get much visible output. To see what our results are, we need to look into the variable. The simplest way is to call `subjectresults` directly:

In [4]:
subjectresults


            QueryID: d1fc6c8c-8b64-42cf-a372-9a7a53a0a151
            Query:SELECT results.* EXCEPT(rn) FROM (SELECT ROW_NUMBER() OVER (PARTITION BY all_Subjects_v3_0_w_RS.id) as rn, all_Subjects_v3_0_w_RS.id AS id, all_Subjects_v3_0_w_RS.identifier AS identifier, all_Subjects_v3_0_w_RS.species AS species, all_Subjects_v3_0_w_RS.sex AS sex, all_Subjects_v3_0_w_RS.race AS race, all_Subjects_v3_0_w_RS.ethnicity AS ethnicity, all_Subjects_v3_0_w_RS.days_to_birth AS days_to_birth, all_Subjects_v3_0_w_RS.subject_associated_project AS subject_associated_project, all_Subjects_v3_0_w_RS.vital_status AS vital_status, all_Subjects_v3_0_w_RS.age_at_death AS age_at_death, all_Subjects_v3_0_w_RS.cause_of_death AS cause_of_death FROM gdc-bq-sample.dev.all_Subjects_v3_0_w_RS AS all_Subjects_v3_0_w_RS LEFT JOIN UNNEST(all_Subjects_v3_0_w_RS.ResearchSubject) AS _ResearchSubject WHERE (UPPER(_ResearchSubject.primary_diagnosis_site) = UPPER('brain'))) as results WHERE rn = 1
            Offset: 0
      

This output tells us our QueryID, which we don't really need, but the computer does to track our questions. Then it tells us four parameters that describe our results:

---

- **Offset:** This is how many rows of information we've told the query to skip in the data, here we didn't tell it to skip anything, so the offset is zero
- **Count:** This is how many rows the current page of our results table has. To keep searches fast, we default to pages with 100 rows.
- **Total Row Count:** This is how many rows are in the full results table
- **More pages:** This is alwasys a True or False. False means that our current page has all the availble results. True means that we will see only the first 100 results in this table, and will need to page through for more.

---
    
Now that we've seen the metadata about our results, let's look at the actual table. The easiest way to do this is by using the python function `.to_dataframe()` on our `subjectresults` variable:

In [5]:
subjectresults.to_dataframe()

,id,identifier,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,age_at_death,cause_of_death
0,ACRIN-DSC-MR-Brain-102,"[{'system': 'IDC', 'value': 'ACRIN-DSC-MR-Brai...",Homo sapiens,None,None,None,None,[acrin_dsc_mr_brain],None,None,None
1,ACRIN-FMISO-Brain-012,"[{'system': 'IDC', 'value': 'ACRIN-FMISO-Brain...",Homo sapiens,None,None,None,None,[acrin_fmiso_brain],None,None,None
2,C19680,"[{'system': 'PDC', 'value': 'C19680'}]",Homo sapiens,female,black or african american,not hispanic or latino,None,[Proteogenomic Analysis of Pediatric Brain Can...,Alive,None,Not Reported
3,C3N-01814,"[{'system': 'GDC', 'value': 'C3N-01814'}, {'sy...",Homo sapiens,female,not reported,not reported,-17813,"[CPTAC3-Discovery, CPTAC-3]",Alive,None,None
4,GENIE-GRCC-6563ba26,"[{'system': 'GDC', 'value': 'GENIE-GRCC-6563ba...",Homo sapiens,male,not allowed to collect,not allowed to collect,-15705,[GENIE-GRCC],Not Reported,None,None
...,...,...,...,...,...,...,...,...,...,...,...
95,TCGA-06-0240,"[{'system': 'GDC', 'value': 'TCGA-06-0240'}, {...",Homo sapiens,male,white,not hispanic or latino,-20937,"[tcga_gbm, TCGA-GBM]",Dead,621,None
96,TCGA-12-1092,"[{'system': 'GDC', 'value': 'TCGA-12-1092'}, {...",Homo sapiens,male,white,not hispanic or latino,-21411,"[tcga_gbm, TCGA-GBM]",Dead,661,None
97,TCGA-19-1386,"[{'system': 'GDC', 'value': 'TCGA-19-1386'}, {...",Homo sapiens,male,white,not hispanic or latino,-18678,"[tcga_gbm, TCGA-GBM]",Dead,276,None
98,TCGA-26-1442,"[{'system': 'GDC', 'value': 'TCGA-26-1442'}]",Homo sapiens,male,white,not hispanic or latino,-15950,[TCGA-GBM],Alive,None,None


By default `to_dataframe()` shows us the first and last five rows for the first page of our results, so we can easily preview our data.

Since we queried the Subject endpoint, our default results tell us Subject level information, that is, information about unique individuals: their sex, race, age, species, etc. The `id` column tells us the unique identifier for each individual. The identifier column has nested information about what study or studies a Subject participated in, and will list all of their research_subject identifiers. 

<span style="background-color:#f20505;color:#f5f5f5"> Need to add subject ID info to output. Devs working on it</span>


### research_subject

But, if we're interested in what research_subjects meet our critera, we can also run our query against the research_subject endpoint:

In [14]:
myquery.subject.files.run().to_dataframe()

Getting results from database

Total execution time: 21144 ms


,id,identifier,label,data_category,data_type,file_format,associated_project,drs_uri,byte_size,checksum,data_modality,imaging_modality,dbgap_accession_number
0,00080a96-0151-4b11-8da9-4b98a938cb73,"[{'system': 'IDC', 'value': '00080a96-0151-4b1...",idc/00080a96-0151-4b11-8da9-4b98a938cb73.dcm,Imaging,None,DICOM,tcga_gbm,drs://dg.4DFC:00080a96-0151-4b11-8da9-4b98a938...,None,None,Imaging,MR,None
1,003187ea-900e-46c3-b92c-1aafd2a01c65,"[{'system': 'IDC', 'value': '003187ea-900e-46c...",idc/003187ea-900e-46c3-b92c-1aafd2a01c65.dcm,Imaging,None,DICOM,ivygap,drs://dg.4DFC:003187ea-900e-46c3-b92c-1aafd2a0...,None,None,Imaging,MR,None
2,003e68ab-d86e-49eb-a95a-9a2e3c6f950e,"[{'system': 'IDC', 'value': '003e68ab-d86e-49e...",idc/003e68ab-d86e-49eb-a95a-9a2e3c6f950e.dcm,Imaging,None,DICOM,tcga_gbm,drs://dg.4DFC:003e68ab-d86e-49eb-a95a-9a2e3c6f...,None,None,Imaging,MR,None
3,004fc633-b49d-4e45-a40f-8c81dbf679a2,"[{'system': 'IDC', 'value': '004fc633-b49d-4e4...",idc/004fc633-b49d-4e45-a40f-8c81dbf679a2.dcm,Imaging,None,DICOM,acrin_fmiso_brain,drs://dg.4DFC:004fc633-b49d-4e45-a40f-8c81dbf6...,None,None,Imaging,MR,None
4,0093f695-6ba4-4406-b6cf-7e4986d5e0a5,"[{'system': 'IDC', 'value': '0093f695-6ba4-440...",idc/0093f695-6ba4-4406-b6cf-7e4986d5e0a5.dcm,Imaging,None,DICOM,cptac_gbm,drs://dg.4DFC:0093f695-6ba4-4406-b6cf-7e4986d5...,None,None,Imaging,MR,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,060e7acd-53ff-49e3-b26a-94a70ae702c8,"[{'system': 'IDC', 'value': '060e7acd-53ff-49e...",idc/060e7acd-53ff-49e3-b26a-94a70ae702c8.dcm,Imaging,None,DICOM,acrin_dsc_mr_brain,drs://dg.4DFC:060e7acd-53ff-49e3-b26a-94a70ae7...,None,None,Imaging,MR,None
96,06174486-dfe5-439b-86d5-05e8c7f68461,"[{'system': 'IDC', 'value': '06174486-dfe5-439...",idc/06174486-dfe5-439b-86d5-05e8c7f68461.dcm,Imaging,None,DICOM,acrin_dsc_mr_brain,drs://dg.4DFC:06174486-dfe5-439b-86d5-05e8c7f6...,None,None,Imaging,MR,None
97,061cb26a-3020-4b46-b295-8bfced81e542,"[{'system': 'IDC', 'value': '061cb26a-3020-4b4...",idc/061cb26a-3020-4b46-b295-8bfced81e542.dcm,Imaging,None,DICOM,ivygap,drs://dg.4DFC:061cb26a-3020-4b46-b295-8bfced81...,None,None,Imaging,MR,None
98,062c9841-aff4-4519-9bc9-3deab664f5ce,"[{'system': 'GDC', 'value': '062c9841-aff4-451...",27e86e11-a91a-4a57-a421-e890ea7eced1_wxs_gdc_r...,Sequencing Reads,Aligned Reads,BAM,CPTAC-3,drs://dg.4DFC:062c9841-aff4-4519-9bc9-3deab664...,34796094421,34e47571fd104086abb4ec93b08796b4,Genomic,None,phs001287


In [6]:
resubjectresults = myquery.research_subject.run()
resubjectresults

Getting results from database

Total execution time: 7601 ms



            QueryID: d347efd6-39a1-4837-a5cf-c856eda0d323
            Query:SELECT results.* EXCEPT(rn) FROM (SELECT ROW_NUMBER() OVER (PARTITION BY _ResearchSubject.id) as rn, _ResearchSubject.id AS id, _ResearchSubject.identifier AS identifier, _ResearchSubject.member_of_research_project AS member_of_research_project, _ResearchSubject.primary_diagnosis_condition AS primary_diagnosis_condition, _ResearchSubject.primary_diagnosis_site AS primary_diagnosis_site, all_Subjects_v3_0_w_RS.id AS subject_id FROM gdc-bq-sample.dev.all_Subjects_v3_0_w_RS AS all_Subjects_v3_0_w_RS INNER JOIN UNNEST(all_Subjects_v3_0_w_RS.ResearchSubject) AS _ResearchSubject WHERE (UPPER(_ResearchSubject.primary_diagnosis_site) = UPPER('brain'))) as results WHERE rn = 1
            Offset: 0
            Count: 100
            Total Row Count: 2923
            More pages: True
            

Now we see that our 2314 subjects have 2923 research_subjects between them, that means that some, but not all, of our subjects were participants in more than one study. Let's peek at the data:

In [7]:
resubjectresults.to_dataframe()

,id,identifier,member_of_research_project,primary_diagnosis_condition,primary_diagnosis_site,subject_id
0,0906ddde-02fa-417d-927b-1c400b189cac,"[{'system': 'GDC', 'value': '0906ddde-02fa-417...",TCGA-LGG,Gliomas,Brain,TCGA-HT-8114
1,1f13065a-40f7-455d-b8eb-f9a128722eac,"[{'system': 'GDC', 'value': '1f13065a-40f7-455...",TCGA-LGG,Gliomas,Brain,TCGA-P5-A5EX
2,2625006e-fa42-434c-a64d-cf8a75ab1ede,"[{'system': 'GDC', 'value': '2625006e-fa42-434...",GENIE-GRCC,"Neoplasms, NOS",Brain,GENIE-GRCC-26e041c1
3,3ecb27b2-75cc-44d0-879e-035e19f39193,"[{'system': 'GDC', 'value': '3ecb27b2-75cc-44d...",GENIE-MSK,Germ Cell Neoplasms,Brain,GENIE-MSK-P-0002849
4,46a02e71-b779-4a08-98d1-99a6c8fc66f2,"[{'system': 'GDC', 'value': '46a02e71-b779-4a0...",GENIE-MSK,"Neoplasms, NOS",Brain,GENIE-MSK-P-0015550
...,...,...,...,...,...,...
95,TCGA-06-0158__tcga_gbm,"[{'system': 'IDC', 'value': 'TCGA-06-0158'}]",tcga_gbm,None,Brain,TCGA-06-0158
96,TCGA-08-0514__tcga_gbm,"[{'system': 'IDC', 'value': 'TCGA-08-0514'}]",tcga_gbm,None,Brain,TCGA-08-0514
97,TCGA-19-2621__tcga_gbm,"[{'system': 'IDC', 'value': 'TCGA-19-2621'}]",tcga_gbm,None,Brain,TCGA-19-2621
98,TCGA-76-6191__tcga_gbm,"[{'system': 'IDC', 'value': 'TCGA-76-6191'}]",tcga_gbm,None,Brain,TCGA-76-6191


Each row from the research-subject endpoint results tells us about a subject in a given study. Any given subject will have one row per study they participated in. Using this endpoint we can find out information like what studies fit our search criteria, and also get data that we can filter to have only subjects from multiple studies, or only subjects from single studies.

### specimens

We can use this same query to see what specimens are available for brain tissue at the CDA:

In [8]:
specresults =  myquery.specimen.run()
print(specresults)

Getting results from database

Total execution time: 7610 ms

            QueryID: cf1c2093-9666-4270-83b4-c461f96fa49c
            Query:SELECT results.* EXCEPT(rn) FROM (SELECT ROW_NUMBER() OVER (PARTITION BY _ResearchSubject_Specimen.id) as rn, _ResearchSubject_Specimen.id AS id, _ResearchSubject_Specimen.identifier AS identifier, _ResearchSubject_Specimen.associated_project AS associated_project, _ResearchSubject_Specimen.age_at_collection AS age_at_collection, _ResearchSubject_Specimen.primary_disease_type AS primary_disease_type, _ResearchSubject_Specimen.anatomical_site AS anatomical_site, _ResearchSubject_Specimen.source_material_type AS source_material_type, _ResearchSubject_Specimen.specimen_type AS specimen_type, _ResearchSubject_Specimen.derived_from_specimen AS derived_from_specimen, _ResearchSubject_Specimen.derived_from_subject AS derived_from_subject, all_Subjects_v3_0_w_RS.id AS subject_id, _ResearchSubject.id AS researchsubject_id FROM gdc-bq-sample.dev.all_Subjects_v3_0_w_RS AS all_Subjects_v3_0_w

Nearly 40,000 specimens meet our search criteria! We would typically expect this number to be much larger than our number of subjects or research_subjects. First because studies will often take more than one sample per subject, and second because any given specimen might be aliquoted out to be used in multiple tests. Since we didn't specify any further filters, our results will return all of these as seperate speciments. Let's look at a few:

In [9]:
specresults.to_dataframe()

,id,identifier,associated_project,age_at_collection,primary_disease_type,anatomical_site,source_material_type,specimen_type,derived_from_specimen,derived_from_subject,subject_id,researchsubject_id
0,01bc8fc0-4b19-52ef-b7ea-693a84fc8f4b,"[{'system': 'GDC', 'value': '01bc8fc0-4b19-52e...",CPTAC-3,-14468,Gliomas,None,Blood Derived Normal,analyte,6e6f946b-8fe0-5a69-b7f3-92b5da4ae249,C3N-00663,C3N-00663,a23aef16-6b33-405f-99dc-d4e9976dbe6e
1,02719fb3-2c30-4afa-b0e1-b51654332bdb,"[{'system': 'GDC', 'value': '02719fb3-2c30-4af...",TCGA-LGG,-12669,Gliomas,None,Primary Tumor,aliquot,2ba59315-5c36-4c5b-b226-1399a1571c05,TCGA-HT-8019,TCGA-HT-8019,9d93834a-3bf0-41fd-8c74-5ac49a6e78cd
2,0275d27c-e387-4ebe-8118-972f4190493d,"[{'system': 'GDC', 'value': '0275d27c-e387-4eb...",TCGA-GBM,-31627,Gliomas,None,Blood Derived Normal,analyte,4c688bfc-794a-4480-970b-2456586dc6b9,TCGA-06-0126,TCGA-06-0126,d88bbd87-e876-4a44-96f5-c28ceac661b8
3,02bcf082-e661-5775-90bf-d3e0feefd428,"[{'system': 'GDC', 'value': '02bcf082-e661-577...",TCGA-LGG,-6857,Gliomas,None,Primary Tumor,portion,abe98b18-1be6-4023-a4ed-8f338fd3ebd2,TCGA-HT-7482,TCGA-HT-7482,448d3e3b-622c-4e79-bb1d-b5f2e92970e8
4,03209a36-67a0-48df-a9f7-a0cedd0db82f,"[{'system': 'GDC', 'value': '03209a36-67a0-48d...",TCGA-GBM,-23952,Gliomas,None,Blood Derived Normal,portion,873b9fe3-27e5-47d1-9149-a28c4faac54a,TCGA-12-0653,TCGA-12-0653,a7b55dae-c9b8-45c6-986f-fb1055f1c67b
...,...,...,...,...,...,...,...,...,...,...,...,...
95,42760c1d-081d-42bd-829f-a97370cec1fa,"[{'system': 'GDC', 'value': '42760c1d-081d-42b...",TCGA-GBM,-17080,Gliomas,None,Primary Tumor,sample,initial specimen,TCGA-02-0099,TCGA-02-0099,883dc176-925a-44f7-9ec8-e0cce33c2a54
96,42ddba08-aff3-4807-bd13-25ac29196fd2,"[{'system': 'GDC', 'value': '42ddba08-aff3-480...",TCGA-LGG,-21450,Gliomas,None,Primary Tumor,portion,79ea61fa-28f3-4598-8415-ae79b177d1aa,TCGA-FG-A4MU,TCGA-FG-A4MU,7662940a-22de-44d8-b59a-2f2b090ea6d5
97,42e0c08f-a503-530d-ab67-99915ac35b13,"[{'system': 'GDC', 'value': '42e0c08f-a503-530...",CPTAC-3,-22583,Gliomas,None,Primary Tumor,analyte,77020759-329b-5944-8bf6-027e0dc20722,C3L-03266,C3L-03266,f8ced7cc-1962-4825-b63e-da8a1fb42cf2
98,432e4c59-3e2b-4dba-9f79-4074ee8a066b,"[{'system': 'GDC', 'value': '432e4c59-3e2b-4db...",TCGA-GBM,-18657,Gliomas,None,Primary Tumor,analyte,3461a96f-e609-4c6e-9a87-9e6ec7131396,TCGA-02-0116,TCGA-02-0116,e117fe84-f003-40a4-96b5-8e26d0513151


### file
<span style="background-color:#f20505;color:#f5f5f5"> This command does not work right now. Devs working on it</span>

The file endpoint returns information about files that meet our search criteria, regardless of whether they are attached to subjects, research-subjects or specimens: 

In [11]:
myquery.files.run()

Getting results from database

Total execution time: 35099 ms



            QueryID: a46352ce-8755-4429-a9b7-6dc5d2967ef8
            Query:SELECT results.* EXCEPT(rn) FROM (SELECT ROW_NUMBER() OVER (PARTITION BY results.id) as rn, results.* FROM (SELECT results.* EXCEPT(rn) FROM (SELECT ROW_NUMBER() OVER (PARTITION BY all_Files_v3_0_w_RS.id) as rn, all_Files_v3_0_w_RS.id AS id, all_Files_v3_0_w_RS.identifier AS identifier, all_Files_v3_0_w_RS.label AS label, all_Files_v3_0_w_RS.data_category AS data_category, all_Files_v3_0_w_RS.data_type AS data_type, all_Files_v3_0_w_RS.file_format AS file_format, all_Files_v3_0_w_RS.associated_project AS associated_project, all_Files_v3_0_w_RS.drs_uri AS drs_uri, all_Files_v3_0_w_RS.byte_size AS byte_size, all_Files_v3_0_w_RS.checksum AS checksum, all_Files_v3_0_w_RS.data_modality AS data_modality, all_Files_v3_0_w_RS.imaging_modality AS imaging_modality, all_Files_v3_0_w_RS.dbgap_accession_number AS dbgap_accession_number FROM gdc-bq-sample.dev.all_Subjects_v3_0_w_RS AS all_Subjects_v3_0_w_RS INNER JOIN UNNES

In [13]:
fileresults = myquery.files.run()
fileresults.to_dataframe()

Getting results from database

Total execution time: 3924 ms


,id,identifier,label,data_category,data_type,file_format,associated_project,drs_uri,byte_size,checksum,data_modality,imaging_modality,dbgap_accession_number
0,00011c37-d1de-4adb-81e4-07494b3846cc,"[{'system': 'IDC', 'value': '00011c37-d1de-4ad...",idc/00011c37-d1de-4adb-81e4-07494b3846cc.dcm,Imaging,None,DICOM,qin_gbm_treatment_response,drs://dg.4DFC:00011c37-d1de-4adb-81e4-07494b38...,None,None,Imaging,MR,None
1,000f2ec5-f19b-4f93-a376-6f72f677a059,"[{'system': 'IDC', 'value': '000f2ec5-f19b-4f9...",idc/000f2ec5-f19b-4f93-a376-6f72f677a059.dcm,Imaging,None,DICOM,acrin_dsc_mr_brain,drs://dg.4DFC:000f2ec5-f19b-4f93-a376-6f72f677...,None,None,Imaging,MR,None
2,001024d1-9900-4544-937f-fd67fff7883e,"[{'system': 'IDC', 'value': '001024d1-9900-454...",idc/001024d1-9900-4544-937f-fd67fff7883e.dcm,Imaging,None,DICOM,acrin_fmiso_brain,drs://dg.4DFC:001024d1-9900-4544-937f-fd67fff7...,None,None,Imaging,MR,None
3,00105c94-2c0a-4468-ac53-5f34ba574429,"[{'system': 'IDC', 'value': '00105c94-2c0a-446...",idc/00105c94-2c0a-4468-ac53-5f34ba574429.dcm,Imaging,None,DICOM,acrin_fmiso_brain,drs://dg.4DFC:00105c94-2c0a-4468-ac53-5f34ba57...,None,None,Imaging,MR,None
4,001573bb-e6bb-4267-91d5-5c7b650a4e0a,"[{'system': 'IDC', 'value': '001573bb-e6bb-426...",idc/001573bb-e6bb-4267-91d5-5c7b650a4e0a.dcm,Imaging,None,DICOM,acrin_dsc_mr_brain,drs://dg.4DFC:001573bb-e6bb-4267-91d5-5c7b650a...,None,None,Imaging,MR,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,01eb0845-bddc-463b-b49d-5f3c50b6a725,"[{'system': 'IDC', 'value': '01eb0845-bddc-463...",idc/01eb0845-bddc-463b-b49d-5f3c50b6a725.dcm,Imaging,None,DICOM,rembrandt,drs://dg.4DFC:01eb0845-bddc-463b-b49d-5f3c50b6...,None,None,Imaging,MR,None
96,01fb9c85-cd5a-49a6-ae4a-f6c9d5cb4cbe,"[{'system': 'IDC', 'value': '01fb9c85-cd5a-49a...",idc/01fb9c85-cd5a-49a6-ae4a-f6c9d5cb4cbe.dcm,Imaging,None,DICOM,ivygap,drs://dg.4DFC:01fb9c85-cd5a-49a6-ae4a-f6c9d5cb...,None,None,Imaging,MR,None
97,01fead59-6b65-4076-a0a5-82affcd8f6a2,"[{'system': 'IDC', 'value': '01fead59-6b65-407...",idc/01fead59-6b65-4076-a0a5-82affcd8f6a2.dcm,Imaging,None,DICOM,tcga_lgg,drs://dg.4DFC:01fead59-6b65-4076-a0a5-82affcd8...,None,None,Imaging,MR,None
98,01fefebf-867b-4b74-89dd-b2f168e39363,"[{'system': 'IDC', 'value': '01fefebf-867b-4b7...",idc/01fefebf-867b-4b74-89dd-b2f168e39363.dcm,Imaging,None,DICOM,rembrandt,drs://dg.4DFC:01fefebf-867b-4b74-89dd-b2f168e3...,None,None,Imaging,MR,None


As you might expect, searching file gives us a huge number of results. This is great if you are surveying what kind of data is available, but is less useful for getting a coherent cohort. 

A better way to get files for a specific cohort is to chain your queries together, which we cover in the next tutorial [Advanced Search](../AdvancedSearch-Chaining): Combine information from multiple endpoints, and build And/Or/Like and other advanced query strings


What are all these fields?

---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<ul>
  <li><b>id:</b> The unique identifier for this file</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the file had there</li>
  <li><b>label:</b> The full name of the file</li>
  <li><b>data_catagory:</b> A desecription of the kind of general kind data the file holds</li>
  <li><b>data_type:</b> A more specific descripton of the data type</li>
  <li><b>file_format:</b> The extension of the file</li>
  <li><b>associated_project:</b> The name the data center uses for the study this file was generated for</li>
  <li><b>drs_uri:</b> A unique identifier that can be used to retreive this specific file from a server</li>
  <li><b>byte_size:</b> Size of the file in bytes</li>
  <li><b>checksum:</b> The md5 value for the file</li>
  <li><b>data_modality:</b> A high level descriptor of file data, always one of "Genomic", "Proteomic", or "Imaging"</li>
  <li><b>imaging_modality</b> For files with the `data_modality` of "Imaging", a descriptor for the image type</li>
  <li><b>dbgap_accession_number:</b></li>
</ul>  

</div>
    
---